# ESMfold embedding

> A collection of plot functions

In [ ]:
#| default_exp esm

In [ ]:
#| hide
from nbdev.showdoc import *
%matplotlib inline

In [ ]:
#| export
from fastbook import *
from fairscale.nn.data_parallel import FullyShardedDataParallel as FSDP
from fairscale.nn.wrap import enable_wrap, wrap
import esm
from tqdm.notebook import tqdm; tqdm.pandas()
import gc

In [ ]:
#| export
def esm_embeddings(df: pd.DataFrame, colname: str, model_name: str = "esm2_t33_650M_UR50D") -> pd.DataFrame:
    """
    Extract ESM embeddings from a DataFrame using the specified ESM model and feature extraction function.
    
    Parameters:
    -----------
    df: pd.DataFrame
        The DataFrame containing the sequences.
    colname: str
        The name of the column containing the sequences.
    model_name: str, optional (default="esm2_t33_650M_UR50D")
        The name of the ESM model to use for the embeddings.
    
    Returns:
    --------
    pd.DataFrame
        The DataFrame containing the ESM embeddings.
    """
    
    # Initialize distributed world with world_size 1
    url = "tcp://localhost:23456"
    torch.distributed.init_process_group(backend="nccl", init_method=url, world_size=1, rank=0)

    # Download model data from the hub
    model_data, regression_data = esm.pretrained._download_model_and_regression_data(model_name)

    # Initialize the model with FSDP wrapper
    fsdp_params = dict(
        mixed_precision=True,
        flatten_parameters=True,
        state_dict_device=torch.device("cpu"),  # reduce GPU mem usage
        cpu_offload=True,  # enable cpu offloading
    )

    with enable_wrap(wrapper_cls=FSDP, **fsdp_params):
        model, vocab = esm.pretrained.load_model_and_alphabet_core(
            model_name, model_data, regression_data
        )
        batch_converter = vocab.get_batch_converter()
        model.eval()

        # Wrap each layer in FSDP separately
        for name, child in model.named_children():
            if name == "layers":
                for layer_name, layer in child.named_children():
                    wrapped_layer = wrap(layer)
                    setattr(child, layer_name, wrapped_layer)
        model = wrap(model)

        # Define the feature extraction function
        def get_feature(r, colname=colname) -> np.ndarray:
            data = [('protein', r[colname])]
            labels, strs, tokens = batch_converter(data)
            with torch.no_grad():
                results = model(tokens.cuda(), repr_layers=[33], return_contacts=True)
            rpr = results["representations"][33].squeeze()
            rpr = rpr[1 : len(r[colname]) + 1].mean(0).detach().cpu().numpy()

            del results, labels, strs, tokens, data #especially need to delete those on cuda: tokens, results
            gc.collect()

            return rpr
        
        # Apply the feature extraction function to each row in the DataFrame
        series = df.apply(get_feature, axis=1)
        df_feature = pd.DataFrame(series.tolist())

        return df_feature

In [ ]:
df = pd.read_csv('kras_seq.csv')

In [ ]:
df = df[['ID','g12d_seq']]

In [ ]:
feature = esm_embeddings(df,'g12d_seq')

RuntimeError: ProcessGroupNCCL is only supported with GPUs, no GPUs found!

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()